In [ ]:
# Data Science Final Project
#
# Development of vacant lots adjacent to Chicago public transit station.
#
# Greg Krajenta
#
# 


In [1]:
#Foundational Methodology
#
#
#1. What is the problem you are trying to solve?
#     In Chicago, IL, the city owns two vacant lots adjacent to a public train station and bus depot. 
#     These lots are in need of development, but a suitable business has not been found.
#
# 2. How can the data solve this problem?
#     I will examine what venues are adjacent to stops for the whole of the train system, and for the subset most similar to the Harlem North Branch Blue Line stop.
#     I am applying the assumption that the most common existing venue types are the most likely to be successful. 
#
# 3. What data will you need?
#     I will need the latitude and longitude of all CTA train stations, and venue location data throughought Chicago.
#     I may also want to examine other factors, like the presence of a highway entrance, how many buses have a stop there, stations in similar neighborhoods, and stations only along 
#     this branch of the blue line.
# 
# 4. What are your data sources and how will you get it?
#     I will use Foursquare for the venue location data, and a CSV provided by the City of Chicago for the train station locations.
#
# 5. Is the data you collected representative of the problem to be solved?
#     Yes.
#
# 6. What additional work is required to harmonize the data?
#     One-hot encoding for categories.
#     Simplify CTA_geo_data_df to remove unwanted columns & duplicate stops.
#     Drop rows for the actual CTA station venue.
#
# 7. How can the data be visualized to get the answer?
#      #####TBD#### 
#
# 8. Does the model used actually answer the question, or does it need to be ajusted?
#     ####TBD#####
#
# 9. Can you put the model into practice?
#       ###TBD###
# 
# 10. Can you get constructive feedback into your answer?
#     #####TBD######
#
#

In [35]:
# Import all tools needed for this project.

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import numpy as np
#from sklearn.metrics import jaccard_score

import json # library to handle JSON files
import requests # library to handle requests
from pandas import json_normalize # tranform JSON file into a pandas dataframe

#!conda install -c conda-forge geopy --yes
#from geopy.geocoders import Nominatim

# Matplotlib and associated plotting modules
#import matplotlib.cm as cm
#import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#import folium # map rendering library


print('Libraries imported,beautiful.')

Libraries imported,beautiful.


In [2]:
#Create dataframe with CTA station entrances.


CTA_geo_data_df = pd.read_csv("https://github.com/gkrajenta/Coursera_Capstone/blob/67a0fdbe97c40897c26cf786369a692994c475c8/CTA_-_System_Information_-_List_of__L__Stops.csv?raw=true")
CTA_geo_data_df.head()
#CTA_geo_data_df.shape
#CTA_geo_data_df.describe

#Knowledge: Add  ?raw=true  to end of Github permalink in order to read the csv into pandas df

,STOP_ID,DIRECTION_ID,STOP_NAME,STATION_NAME,STATION_DESCRIPTIVE_NAME,MAP_ID,ADA,RED,BLUE,G,BRN,P,Pexp,Y,Pnk,O,Location
0,30162,W,18th (54th/Cermak-bound),18th,18th (Pink Line),40830,True,False,False,False,False,False,False,False,True,False,"(41.857908, -87.669147)"
1,30161,E,18th (Loop-bound),18th,18th (Pink Line),40830,True,False,False,False,False,False,False,False,True,False,"(41.857908, -87.669147)"
2,30022,N,35th/Archer (Loop-bound),35th/Archer,35th/Archer (Orange Line),40120,True,False,False,False,False,False,False,False,False,True,"(41.829353, -87.680622)"
3,30023,S,35th/Archer (Midway-bound),35th/Archer,35th/Archer (Orange Line),40120,True,False,False,False,False,False,False,False,False,True,"(41.829353, -87.680622)"
4,30214,S,35-Bronzeville-IIT (63rd-bound),35th-Bronzeville-IIT,35th-Bronzeville-IIT (Green Line),41120,True,False,False,True,False,False,False,False,False,False,"(41.831677, -87.625826)"


In [3]:
CTA_geo_data_df[['Latitude', 'Longitude']] = CTA_geo_data_df['Location'].str.split(' ', 1, expand=True)
CTA_geo_data_df['Latitude']=CTA_geo_data_df['Latitude'].str.replace("(","")
CTA_geo_data_df['Latitude']=CTA_geo_data_df['Latitude'].str.replace(",","")
CTA_geo_data_df['Longitude']=CTA_geo_data_df['Longitude'].str.replace(")","")

CTA_geo_data_df.head()
#CTA_geo_data_df.shape

,STOP_ID,DIRECTION_ID,STOP_NAME,STATION_NAME,STATION_DESCRIPTIVE_NAME,MAP_ID,ADA,RED,BLUE,G,BRN,P,Pexp,Y,Pnk,O,Location,Latitude,Longitude
0,30162,W,18th (54th/Cermak-bound),18th,18th (Pink Line),40830,True,False,False,False,False,False,False,False,True,False,"(41.857908, -87.669147)",41.857908,-87.669147
1,30161,E,18th (Loop-bound),18th,18th (Pink Line),40830,True,False,False,False,False,False,False,False,True,False,"(41.857908, -87.669147)",41.857908,-87.669147
2,30022,N,35th/Archer (Loop-bound),35th/Archer,35th/Archer (Orange Line),40120,True,False,False,False,False,False,False,False,False,True,"(41.829353, -87.680622)",41.829353,-87.680622
3,30023,S,35th/Archer (Midway-bound),35th/Archer,35th/Archer (Orange Line),40120,True,False,False,False,False,False,False,False,False,True,"(41.829353, -87.680622)",41.829353,-87.680622
4,30214,S,35-Bronzeville-IIT (63rd-bound),35th-Bronzeville-IIT,35th-Bronzeville-IIT (Green Line),41120,True,False,False,True,False,False,False,False,False,False,"(41.831677, -87.625826)",41.831677,-87.625826


In [4]:
#Remove duplicate stations / lat&lon


CTA_stations_unique_df = CTA_geo_data_df.copy()
CTA_stations_unique_df.drop_duplicates(subset=['Location'],inplace=True)

CTA_stations_unique_df.head()
#CTA_stations_unique_df.shape

,STOP_ID,DIRECTION_ID,STOP_NAME,STATION_NAME,STATION_DESCRIPTIVE_NAME,MAP_ID,ADA,RED,BLUE,G,BRN,P,Pexp,Y,Pnk,O,Location,Latitude,Longitude
0,30162,W,18th (54th/Cermak-bound),18th,18th (Pink Line),40830,True,False,False,False,False,False,False,False,True,False,"(41.857908, -87.669147)",41.857908,-87.669147
2,30022,N,35th/Archer (Loop-bound),35th/Archer,35th/Archer (Orange Line),40120,True,False,False,False,False,False,False,False,False,True,"(41.829353, -87.680622)",41.829353,-87.680622
4,30214,S,35-Bronzeville-IIT (63rd-bound),35th-Bronzeville-IIT,35th-Bronzeville-IIT (Green Line),41120,True,False,False,True,False,False,False,False,False,False,"(41.831677, -87.625826)",41.831677,-87.625826
6,30246,S,43rd (63rd-bound),43rd,43rd (Green Line),41270,True,False,False,True,False,False,False,False,False,False,"(41.816462, -87.619021)",41.816462,-87.619021
8,30025,S,51st (63rd-bound),51st,51st (Green Line),40130,True,False,False,True,False,False,False,False,False,False,"(41.80209, -87.618487)",41.80209,-87.618487


In [5]:
#Define Foursquare credentials.

CLIENT_ID = 'AD3VDOPBUDVA33LD4LROSAWTG2XBUGJUXUBLVJ0PBFWIITF3'
CLIENT_SECRET = '5VUJQRYOEDCQELVD3YGOOA25U3QRO1NUIXYL5UELAJI1PA1X' 
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
radius = 500 ### in meters.  USE DIFFERENT VALUES FOR THIS TO CREATE MULTIPLE RESULTS SETS####
latitude = '41.98227'  #Value set to Harlem north branch station
longitude = '-87.8089' #Value set to Harlem north branch station


print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AD3VDOPBUDVA33LD4LROSAWTG2XBUGJUXUBLVJ0PBFWIITF3
CLIENT_SECRET:5VUJQRYOEDCQELVD3YGOOA25U3QRO1NUIXYL5UELAJI1PA1X


In [6]:
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url # display URL


results = requests.get(url).json()

#ASSUMPTION: Foursquare data is complete and correct.

In [7]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


In [8]:
#Filter results for the relevant columns.
venues = results['response']['groups'][0]['items']

#Normalize the data, which this data set probably doesn't need.
nearby_venues = json_normalize(venues)

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng','venue.location.distance',]
nearby_venues = nearby_venues.loc[:, filtered_columns]

#Get category information; runs function defined above
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues

,venue.name,venue.categories,venue.location.lat,venue.location.lng,venue.location.distance
0,Taco Burrito King,Mexican Restaurant,41.981155,-87.806820,212
1,Cozy Corner Cafe,American Restaurant,41.979887,-87.807516,288
2,Dollar Tree,Discount Store,41.980065,-87.806743,303
3,Popeyes Louisiana Kitchen,Fried Chicken Joint,41.978683,-87.806725,437
4,Dunkin',Donut Shop,41.979275,-87.806540,386
5,Wendy’s,Fast Food Restaurant,41.980359,-87.807338,248
6,Shell,Gas Station,41.980832,-87.807384,203
7,Teaser's Pub,Bar,41.980715,-87.805607,322
8,Fried Rice Express,Chinese Restaurant,41.980984,-87.805301,330
9,Downey's Har-Hig,Bar,41.981236,-87.806669,217


In [9]:
#Function to get nearby Foursquare venues for all CTA stations.

def getNearbyVenues(names, latitudes, longitudes, radius=250):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['CTA Station', 
                  'Station Latitude', 
                  'Station Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [10]:
#Get adjacent venue data for each CTA stations; calls function defined above.

LIMIT = 50

CTA_venues = getNearbyVenues(names=CTA_stations_unique_df['STATION_DESCRIPTIVE_NAME'],
                                   latitudes=CTA_stations_unique_df['Latitude'],
                                   longitudes=CTA_stations_unique_df['Longitude'])


18th (Pink Line)
35th/Archer (Orange Line)
35th-Bronzeville-IIT (Green Line)
43rd (Green Line)
51st (Green Line)
54th/Cermak (Pink Line)
69th (Red Line)
79th (Red Line)
95th/Dan Ryan (Red Line)
Adams/Wabash (Brown, Green, Orange, Pink & Purple Lines)
Addison (Brown Line)
Addison (Red Line)
Argyle (Red Line)
87th (Red Line)
Armitage (Brown & Purple Lines)
Ashland (Orange Line)
Ashland/63rd (Green Line)
Austin (Blue Line)
Austin (Green Line)
Belmont (Blue Line)
Belmont (Red, Brown & Purple Lines)
Berwyn (Red Line)
Bryn Mawr (Red Line)
California (Pink Line)
California (Green Line)
Central (Green Line)
Central Park (Pink Line)
Cermak-Chinatown (Red Line)
Central (Purple Line)
Cicero (Pink Line)
Cicero (Blue Line)
Cicero (Green Line)
Clark/Lake (Blue, Brown, Green, Orange, Purple & Pink Lines)
Clinton (Blue Line)
Clinton (Green & Pink Lines)
Conservatory (Green Line)
Cottage Grove (Green Line)
Cumberland (Blue Line)
Damen (Pink Line)
Damen (Blue Line)
Damen (Brown Line)
Davis (Purple Line)

In [48]:
#Foursquare venues include the CTA stations. This removes them from the results.

# Get indexes of rows for Train station venues
indexNames = CTA_venues[CTA_venues['Venue Category'].str.contains('Train')].index

# Delete these row indexes from dataFrame
CTA_venues.drop(indexNames , inplace=True)

#CTA_venues.head(20)
#CTA_venues.shape
#len(CTA_venues['Venue Category'].unique())

283

In [16]:
# one hot encoding
CTA_onehot = pd.get_dummies(CTA_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
CTA_onehot['Station'] = CTA_venues['CTA Station'] 

# move Station column to the first column
fixed_columns = [CTA_onehot.columns[283]] + list(CTA_onehot.columns[0:282])
CTA_onehot = CTA_onehot[fixed_columns]

CTA_onehot.head()
#CTA_onehot.shape




,Station,ATM,Accessories Store,Adult Boutique,African Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Residence Hall,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Currency Exchange,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio,Financial or Legal Service,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Health & Beauty Service,Heliport,Historic Site,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Lawyer,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoor Supply Store,Paper / Office Supplies Store,Park,Pastry Shop,Pawn Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Pet Service,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Smoke Shop,Smoothie Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Storage Facility,Street Art,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Water Park,Waterfront

In [17]:
CTA_grouped = CTA_onehot.groupby('Station').sum().reset_index()    #why does swapping this last bit in and out fix/break the code?; changed mean() to sum(). Dumbass.


In [22]:
CTA_grouped.head()
#CTA_grouped.shape


,Station,ATM,Accessories Store,Adult Boutique,African Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Residence Hall,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Currency Exchange,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio,Financial or Legal Service,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Health & Beauty Service,Heliport,Historic Site,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Lawyer,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoor Supply Store,Paper / Office Supplies Store,Park,Pastry Shop,Pawn Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Pet Service,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Smoke Shop,Smoothie Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Storage Facility,Street Art,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Water Park,Waterfront

In [20]:
#Identify most common venues for each stop.

num_top_venues = 10

for station in CTA_grouped['Station']:
     print("----"+station+"----")
     temp = CTA_grouped[CTA_grouped['Station'] == station].T.reset_index()
     temp.columns = ['venue','freq']
     temp = temp.iloc[1:]
     temp['freq'] = temp['freq'].astype(float)
     temp = temp.round({'freq': 2})
     print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
     print('\n')
        
        
#As expected, most results are only one venue category per stop. Use this to identify stops similar to Harlem, but will probably need to compare full venue lists and aggregate totals.

----18th (Pink Line)----
                venue  freq
0  Mexican Restaurant   4.0
1   Food & Drink Shop   1.0
2      Ice Cream Shop   1.0
3            Creperie   1.0
4          Food Truck   1.0
5              Bakery   1.0
6         Record Shop   1.0
7      Discount Store   1.0
8          Donut Shop   1.0
9       Grocery Store   1.0


----35th-Bronzeville-IIT (Green Line)----
                     venue  freq
0           Shipping Store   2.0
1              Pizza Place   1.0
2                     Bank   1.0
3           Sandwich Place   1.0
4              Coffee Shop   1.0
5                      ATM   0.0
6              Music Venue   0.0
7                Nightclub   0.0
8  New American Restaurant   0.0
9               Nail Salon   0.0


----35th/Archer (Orange Line)----
                    venue  freq
0      Mexican Restaurant   1.0
1             Supermarket   1.0
2  Thrift / Vintage Store   1.0
3              Donut Shop   1.0
4             Video Store   1.0
5             Bus Station   1.0


,Station,ATM,Accessories Store,Adult Boutique,African Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Residence Hall,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Currency Exchange,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio,Financial or Legal Service,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Health & Beauty Service,Heliport,Historic Site,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Lawyer,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoor Supply Store,Paper / Office Supplies Store,Park,Pastry Shop,Pawn Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Pet Service,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Smoke Shop,Smoothie Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Storage Facility,Street Art,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Water Park,Waterfront

In [27]:
#Function to return most common venues into neighborhood dataframe.  Called below.

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [41]:
#Create new dataframe by adding most common venues to neighborhoods.
#Must rerun this cell after k-means clustering to add cluster labels.

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Station']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
station_venues_sorted = pd.DataFrame(columns=columns)
station_venues_sorted['Station'] = CTA_grouped['Station']

for ind in np.arange(CTA_grouped.shape[0]):
    station_venues_sorted.iloc[ind, 1:] = return_most_common_venues(CTA_grouped.iloc[ind, :], num_top_venues)
    
station_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)  #Added to update cluster labels after running k-means clustering.
    
station_venues_sorted.head()

,Cluster Labels,Station,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,3,18th (Pink Line),Mexican Restaurant,Food & Drink Shop,Bakery,Donut Shop,Discount Store,Clothing Store,Record Shop,Food Truck,Grocery Store,Ice Cream Shop
1,3,35th-Bronzeville-IIT (Green Line),Shipping Store,Pizza Place,Sandwich Place,Bank,Coffee Shop,Eye Doctor,Exhibit,Event Space,Ethiopian Restaurant,Falafel Restaurant
2,3,35th/Archer (Orange Line),Video Store,Supermarket,Thrift / Vintage Store,Bus Station,Discount Store,Donut Shop,Mexican Restaurant,Falafel Restaurant,Eye Doctor,Exhibit
3,3,43rd (Green Line),Boutique,Sports Bar,Smoke Shop,Restaurant,Ethiopian Restaurant,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant
4,3,47th (Green Line),American Restaurant,Pizza Place,Burger Joint,Shoe Store,Lounge,Liquor Store,Bus Station,Caribbean Restaurant,Breakfast Spot,Performing Arts Venue


In [40]:
####4. Cluster Neighborhoods

# set number of clusters
kclusters = 5

CTA_grouped_clustering = CTA_grouped.drop('Station', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(CTA_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]



array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3], dtype=int32)

In [46]:
station_venues_sorted

,Cluster Labels,Station,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,3,18th (Pink Line),Mexican Restaurant,Food & Drink Shop,Bakery,Donut Shop,Discount Store,Clothing Store,Record Shop,Food Truck,Grocery Store,Ice Cream Shop
1,3,35th-Bronzeville-IIT (Green Line),Shipping Store,Pizza Place,Sandwich Place,Bank,Coffee Shop,Eye Doctor,Exhibit,Event Space,Ethiopian Restaurant,Falafel Restaurant
2,3,35th/Archer (Orange Line),Video Store,Supermarket,Thrift / Vintage Store,Bus Station,Discount Store,Donut Shop,Mexican Restaurant,Falafel Restaurant,Eye Doctor,Exhibit
3,3,43rd (Green Line),Boutique,Sports Bar,Smoke Shop,Restaurant,Ethiopian Restaurant,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant
4,3,47th (Green Line),American Restaurant,Pizza Place,Burger Joint,Shoe Store,Lounge,Liquor Store,Bus Station,Caribbean Restaurant,Breakfast Spot,Performing Arts Venue
5,3,47th (Red Line),Fast Food Restaurant,Food,Bus Station,Park,Women's Store,Ethiopian Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant
6,3,51st (Green Line),Southern / Soul Food Restaurant,Restaurant,Bike Rental / Bike Share,Business Service,American Restaurant,Women's Store,Event Space,Eastern European Restaurant,Electronics Store,Empanada Restaurant
7,3,54th/Cermak (Pink Line),Sandwich Place,Construction & Landscaping,Breakfast Spot,Mexican Restaurant,Currency Exchange,Cycle Studio,Filipino Restaurant,Cupcake Shop,Fast Food Restaurant,Farmers Market
8,3,63rd (Red Line),Fast Food Restaurant,Gas Station,Women's Store,Event Space,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Exhibit
9,3,69th (Red Line),Candy Store,Boutique,Fast Food Restaurant,Bike Rental / Bike Share,Exhibit,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space


In [58]:

venue_list=CTA_venues["Venue Category"]
venue_list.head()

venue_list.value_counts()

Coffee Shop                        127
Sandwich Place                     100
Pizza Place                         76
Mexican Restaurant                  69
Bar                                 59
Chinese Restaurant                  57
Donut Shop                          50
Hotel                               47
Italian Restaurant                  46
American Restaurant                 44
Fast Food Restaurant                41
Gym                                 38
Bus Station                         38
Theater                             34
Café                                34
Grocery Store                       33
Bakery                              32
Pharmacy                            32
Cosmetics Shop                      31
Sushi Restaurant                    30
Asian Restaurant                    27
Burger Joint                        26
Ice Cream Shop                      26
Thai Restaurant                     26
Clothing Store                      24
Seafood Restaurant       

In [ ]:
# Next Steps:

# CTA_venues is the df after initial processing.
# CTA_grouped is the df after one-hot encoding.
# CTA_venue_counts is the df showing max values after one-hot encoding.


# Identify stations similar to Harlem, and key differences.
# Identify most common venue categories throughout the system. 

# Look at subsets of stops: along the O'Hare branch blue line & ones with a bus station


# Try heat map of venue totals to look for venue type patterns

In [ ]:

 full report consisting of all of the following components (15 marks):

Introduction where you discuss the business problem and who would be interested in this project.
Data where you describe the data that will be used to solve the problem and the source of the data.
Methodology section which represents the main component of the report where you discuss and describe any exploratory data analysis that you did, any inferential statistical testing that you performed, if any, and what machine learnings were used and why.
Results section where you discuss the results.
Discussion section where you discuss any observations you noted and any recommendations you can make based on the results.
Conclusion section where you conclude the report.
3. Your choice of a presentation or blogpost. (10 marks)